In [7]:
import json
from pyfreeling import Analyzer

import inspect
print(inspect.getfile(Analyzer))

import upos_converter as up
import importlib
importlib.reload(up)

import nltk
import pandas
import random


/home/master/anaconda3/lib/python3.5/site-packages/pyfreeling/__init__.py


In [2]:
config_file = 'es.cfg'
analyzer = Analyzer(config=config_file)

In [3]:
def tag_it_all(text):
    o = analyzer.run(text.encode('utf-8'),'flush','noner','nortkcon', output='json', lang='es').decode("utf-8")
    return json.loads("[" + o + "]")
# TODO: No esta muy prolijo usar en analyzer global, habria que pasarlo como parametro

In [129]:
#recibe una sentencia y retorna una lista en formato [(form,ctag),...]
def prepare_list_sentence_for_ner(sentence):
    resultado = []
    for word in sentence:
        if '+' in word['ctag']:
            splitCtag = word['ctag'].split('+') 
            splitLemma = word['lemma'].split('+') 
            for i in range(len(splitCtag)):
                resultado.append((splitLemma[i],splitCtag[i]))
        else:
            resultado.append((word['form'],word['ctag']))
    return resultado

In [8]:
import sys
#ruta a la carpeta de nltk_trainner
sys.path.append('/home/master/Descargas/nltk-trainer-master')

#se importa el tagger y chunker ya entrenados con nltk trainner
tagger = nltk.data.load('taggers.pickle')
chunker = nltk.data.load('naiveBayes.pickle')

In [15]:
#recibe una lista de tuplas [(form,ctag)]
#retorna una lista con el tag ner en formato IOB
def list_to_ner(lista_entrada):
    #se calcula el ner
    str_chunks = chunker.parse(lista_entrada)
    #se convierte a formato IOB
    newList = nltk.chunk.tree2conlltags(str_chunks)
    #se eliminan los campos MISC
    list_of_ners = [list(elem) for elem in newList]
    for i in list_of_ners:
        if 'MISC' in i[2]:
            i[2]='O'
    return list_of_ners

A-Revisar postfunction adjetivo
C-no tiene genero ni numero
V-relative, demonstrative, exclamative, numeral
N-	sobran neclass#nesubclass#degree
P-'numeral', possessornum, possessive no tiene
V- dictada VMP00SF problema
S-las prep no tienen genero ni numero

In [209]:
def json_to_string(lista_entrada):
    ret = ""
    for sent_id, entrada in enumerate(lista_entrada):
        list_tags = prepare_list_sentence_for_ner(entrada['tokens'])
        ner_list = list_to_ner(list_tags)
        nerCounter = 0
        ret += '#Sentence '+ str(sent_id+1) +'\n'
        offset = 1
        for token_id, token in enumerate(entrada['tokens']):
            form = token['form']
            if token['tag'].count('+') > 0:
                #Estamos en una contraccion
                tags = token['tag'].split('+')
                lemmas = token['lemma'].split('+')
                ret += str(token_id+1) + "-"
                sub_ret = "\t" + form + '\t_\t_\t_\t_\t_\t_\t_\t_\n'
                offset += len(tags)-1
                for i in range(len(tags)):
                    lemma = lemmas[i]
                    l = list(tags[i])
                    nuestro = up.upos_converter(l)
                    upostag, feats = up.upos_converter(l)
                    #TODO: en la asignacion de ahi abajo ver que pasa con form
                    pos=token_id + i+1
                    sub_ret += str(pos) + '\t' + lemma  + '\t' + lemma + '\t' + upostag + '\t' + feats + '\t_\t_\t_\t_\t' + ner_list[nerCounter][2] +'\n'
                    nerCounter=nerCounter+1
                #Fin del for, concatenamos el sub resultado
                #offset -= 1 #Sacamos 1 al offset pq el for hace uno de mas
                ret += str(token_id + len(tags))+sub_ret
            else:   
                lemma = token['lemma']
                l = list(token['tag'])
                nuestro = up.upos_converter(l)
                upostag, feats = up.upos_converter(l)
                pos=token_id + offset
                ret += str(pos) + '\t' + form  + '\t' + lemma + '\t' + upostag + '\t' + feats + '\t_\t_\t_\t_\t' +  ner_list[nerCounter][2] +'\n'
                nerCounter=nerCounter+1
        #offset += len(entrada['tokens']) -1
        ret += '\n\n'
    return ret

In [210]:
print(json_to_string(tag_it_all("Pedro es el perro del Juan Antonio es malo.")))

#Sentence 1
1	Pedro	pedro	NOUN	Gender=Masc|Number=Sing	_	_	_	_	B-PER
2	es	ser	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres	_	_	_	_	O
3	el	el	DET	Gender=Masc|Number=Sing|PronType=Art	_	_	_	_	O
4	perro	perro	NOUN	Gender=Masc|Number=Sing	_	_	_	_	B-ORG
5-6	del	_	_	_	_	_	_	_	_
5	de	de	ADP		_	_	_	_	O
6	el	el	DET	Gender=Masc|Number=Sing|PronType=Art	_	_	_	_	O
7	Juan	juan	VERB	Mood=Ind|Number=Plur|Person=3|Tense=Pres	_	_	_	_	B-PER
8	Antonio	antonio	NOUN	Gender=Masc|Number=Sing	_	_	_	_	I-PER
9	es	ser	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres	_	_	_	_	O
10	malo	malo	ADJ	Gender=Masc|Number=Sing	_	_	_	_	O
11	.	.	PUNCT		_	_	_	_	O





In [44]:
corpus = pandas.read_csv("corpus_pj.csv", delimiter=',', skip_blank_lines=True, encoding='utf-8')
corpus = corpus.drop_duplicates()
sentencias = corpus["sentencia"]

In [45]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [206]:
importlib.reload(up)
def show(sentencia=None):
    if type(sentencia) == str:
        texto = sentencia
    else:
        if type(sentencia) == int:
            i = sentencia % len(sentencias)
        else:
            i = random.randrange(len(sentencias))
            print(i)
        texto = sentencias[i]
    try:
        text=json_to_string(tag_it_all(texto))
        #print(text)
        for line in text.split("\n"):
            line = line.split("\t")# Splits on whitespace
            if len(line)==10:
                print('{0[0]:<10}{0[1]:<40}{0[2]:<30}{0[3]:<10}{0[4]:>60}{0[5]:>2}{0[6]:>2}{0[7]:>2}{0[8]:>2}{0[9]:>7}'.format(line))
            else:
                print(line[0])
    except Exception as e:
        print(i)
        raise e
        

# show(15213212312)

In [211]:
show(19159)

#Sentence 1
1         Montevideo                              monte_video                   NOUN                                           Gender=Masc|Number=Sing _ _ _ _  B-LOC
2         ,                                       ,                             PUNCT                                                                  _ _ _ _      O
3         treinta                                 30                            NUM                                                      NumForm=Digit _ _ _ _      O
4         de                                      de                            ADP                                                                    _ _ _ _      O
5         setiembre                               setiembre                     NOUN                                           Gender=Masc|Number=Sing _ _ _ _      O
6         de                                      de                            ADP                                                                    _ _ _ _